In [49]:
%reload_ext autoreload
%autoreload 2

In [50]:
import json
import pandas as pd
import numpy as np
from pathlib import Path

import util
import train_and_predict
import feature_importance
import feature_selection
from data import *

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Read Data

In [51]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
path = Path('G:\My Drive\Shaked\WSPC')

In [52]:
train_genomes_path = path / 'train_genomes.fasta'
train_genomes_metedata_path = path / 'train_genomes_info.csv'

test_genomes_path = path / 'test_genomes.fasta'
test_genomes_metedata_path = path / 'test_genomes_info.csv'

train_dataset = GenomesData(train_genomes_path, train_genomes_metedata_path)
test_dataset = GenomesData(test_genomes_path, test_genomes_metedata_path)

Check split train dataset to train and validation according to insertion date

In [53]:
print('train dataset', len(train_dataset.data))

split = feature_selection.split_by_insertion_date(train_dataset, proportion=0.2)

train_idx, validation_idx = next(split.split())

X_train_raw, X_valid_raw = train_dataset.data[train_idx], train_dataset.data[validation_idx]
y_train, y_valid = train_dataset.y[train_idx], train_dataset.y[validation_idx]

print('train', len(X_train_raw), 'validation', len(X_valid_raw))

train dataset 641
train 513 validation 128


In [54]:
y_valid.value_counts()

1    89
0    39
Name: Label, dtype: int64

In [55]:
DATE_INSERTED = 'Date Inserted'

min_validation_date = min(train_dataset.metadata.iloc[validation_idx, :][DATE_INSERTED])
min_validation_date

Timestamp('2018-11-29 03:53:53.656000+0000', tz='UTC')

In [56]:
max_train_date = max(train_dataset.metadata.iloc[train_idx, :][DATE_INSERTED])
max_train_date

Timestamp('2018-11-29 03:08:47.637000+0000', tz='UTC')

In [57]:
assert min_validation_date > max_train_date

# Feature Selection - best k features according to chi2

## 15% validation

In [58]:
min_val = 50
max_val = 600
inc = 50

k_range = range(min_val, max_val + 1, inc)

split = feature_selection.split_by_insertion_date(train_dataset, proportion=0.15)
grid_search = feature_selection.perform_fs_k_best(train_dataset, k_range, split=split)
print(grid_search.best_params_)

param_name = 'k_best__k'
feature_selection.grid_search_results_to_df(grid_search, param_name)

Best roc_auc score is: 0.9118055555555555
{'k_best__k': 600}


,mean_test_balanced_accuracy,mean_test_roc_auc
param_k_best__k,,
50,0.775,0.865
100,0.811,0.863
150,0.819,0.897
200,0.811,0.896
250,0.806,0.894
300,0.819,0.895
350,0.811,0.898
400,0.811,0.899
450,0.811,0.901


## 20% validation

In [59]:
min_val = 50
max_val = 600
inc = 50

k_range = range(min_val, max_val + 1, inc)

split = feature_selection.split_by_insertion_date(train_dataset, proportion=0.2)
grid_search = feature_selection.perform_fs_k_best(train_dataset, k_range, split=split)
print(grid_search.best_params_)

param_name = 'k_best__k'
feature_selection.grid_search_results_to_df(grid_search, param_name)

Best roc_auc score is: 0.9030538749639873
{'k_best__k': 450}


,mean_test_balanced_accuracy,mean_test_roc_auc
param_k_best__k,,
50,0.832,0.873
100,0.828,0.885
150,0.813,0.882
200,0.815,0.897
250,0.826,0.896
300,0.837,0.896
350,0.837,0.898
400,0.832,0.899
450,0.844,0.903


## 5-fold stratified cross validation

In [60]:
min_val = 50
max_val = 600
inc = 50

k_range = range(min_val, max_val + 1, inc)

kf = StratifiedKFold(n_splits=5)
grid_search = feature_selection.perform_fs_k_best(train_dataset, k_range, split=kf)
print(grid_search.best_params_)

param_name = 'k_best__k'
feature_selection.grid_search_results_to_df(grid_search, param_name)

Best roc_auc score is: 0.9627397654572917
{'k_best__k': 550}


,mean_test_balanced_accuracy,mean_test_roc_auc
param_k_best__k,,
50,0.884,0.934
100,0.889,0.951
150,0.894,0.955
200,0.900,0.958
250,0.904,0.955
300,0.897,0.956
350,0.892,0.958
400,0.899,0.957
450,0.897,0.961


The 5-fold cross validation didn't give consistent results. In my opinion 20% validation set is prefered.

# Feature Selection - select best feature from each cluster

In [61]:
min_val = 0
max_val = 0.6
inc = 0.03
t_range = np.arange(min_val, max_val, inc)

# created corr matrix using feature_selection.create_corr_matrix(X_train_raw, y_train) 
corr_matrix_path = path / 'X_train_corr_mat_0.8_k450.csv'
corr_matrix_train = pd.read_csv(corr_matrix_path, index_col=0)
dist_matrix_train = feature_selection.feature_corr_to_dist_matrix(corr_matrix_train)

split = feature_selection.split_by_insertion_date(train_dataset, proportion=0.2)
grid_search_cluster = feature_selection.perform_fs_clusters(train_dataset, dist_matrix_train, t_range, split=split)
print(grid_search_cluster.best_params_)

param_name = 'cluster__threshold'
feature_selection.grid_search_results_to_df(grid_search_cluster, param_name)

threshold=0.0, selected_features= 450
threshold=0.03, selected_features= 406
threshold=0.06, selected_features= 373
threshold=0.09, selected_features= 329
threshold=0.12, selected_features= 298
threshold=0.15, selected_features= 270
threshold=0.18, selected_features= 244
threshold=0.21, selected_features= 219
threshold=0.24, selected_features= 201
threshold=0.27, selected_features= 181
threshold=0.3, selected_features= 159
threshold=0.32999999999999996, selected_features= 136
threshold=0.36, selected_features= 116
threshold=0.39, selected_features= 95
threshold=0.42, selected_features= 74
threshold=0.44999999999999996, selected_features= 60
threshold=0.48, selected_features= 47
threshold=0.51, selected_features= 37
threshold=0.54, selected_features= 30
threshold=0.57, selected_features= 18
threshold=0.0, selected_features= 450
Best roc_auc score is: 0.9030538749639873
{'cluster__threshold': 0.0}


,mean_test_balanced_accuracy,mean_test_roc_auc
param_cluster__threshold,,
0.00,0.844,0.903
0.03,0.850,0.899
0.06,0.843,0.898
0.09,0.824,0.898
0.12,0.843,0.901
0.15,0.850,0.901
0.18,0.850,0.903
0.21,0.856,0.898
0.24,0.850,0.900


# Final model

## Perform feature selection process on the entire train dataset, using best parameters

In [62]:
best_k = 450
best_t = 0.18

In [63]:
X_train_raw, X_test_raw = train_dataset.data, test_dataset.data
y_train, y_test = train_dataset.y, test_dataset.y

# created corr matrix using feature_selection.create_corr_matrix(X_train_raw, y_train) 
corr_matrix_path = path / 'X_train_corr_mat_k450.csv'
X_train_corr_matrix = pd.read_csv(corr_matrix_path, index_col=0)
X_train_dist_mat = feature_selection.feature_corr_to_dist_matrix(X_train_corr_matrix)

fs_pipeline = Pipeline(steps=[('vectorize', CountVectorizer(lowercase=False, binary=True)),
                               ('k_best', SelectKBest(score_func=sklearn.feature_selection.chi2, k=best_k)),
                               ('cluster', feature_selection.SelectHierarchicalClustering(X_train_dist_mat, threshold=best_t)),
                               ('rf', RandomForestClassifier(random_state=0))])

fs_pipeline.fit(X_train_raw, y_train)

results = train_and_predict.predict_and_print_results(X_test_raw, y_test, fs_pipeline)

threshold=0.18, selected_features= 250
false_positive: 20,total NHPs: 100
false_negative: 7,total HPs: 106
BAcc: 0.87
sensitivity: 0.93
specificity: 0.8
aupr_auc: 0.96
roc_auc: 0.95


## Extract final features

In [64]:
all_pgfams = pd.array(fs_pipeline['vectorize'].get_feature_names())
k_best_pgfams = all_pgfams[fs_pipeline['k_best'].get_support()]
cluster_pgfams = k_best_pgfams[fs_pipeline['cluster'].get_support()]

In [65]:
vectorizer = CountVectorizer(lowercase=False, binary=True, vocabulary=cluster_pgfams)
vectorizer.fit_transform(X_train_raw)

<641x250 sparse matrix of type '<class 'numpy.int64'>'
	with 41335 stored elements in Compressed Sparse Row format>

In [66]:
assert all(cluster_pgfams == vectorizer.get_feature_names())

## Final WSPC model
A random forest model, which includes the selected genes as features

In [67]:
WSPC, results = train_and_predict.train_and_predict(X_train_raw, y_train, X_test_raw, y_test, features=cluster_pgfams)

false_positive: 20,total NHPs: 100
false_negative: 7,total HPs: 106
BAcc: 0.87
sensitivity: 0.93
specificity: 0.8
aupr_auc: 0.96
roc_auc: 0.95


# Feature Importance

In [68]:
vectorizer = CountVectorizer(lowercase=False, binary=True, vocabulary=cluster_pgfams)

X_train = train_dataset.vectorize_data(vectorizer)
y_train = train_dataset.y

X_train.head()

,PGF_00003251,PGF_00004164,PGF_00004169,PGF_00004337,PGF_00006100,PGF_00006245,PGF_00006320,PGF_00006377,PGF_00006380,PGF_00007119,...,PGF_10368922,PGF_10424554,PGF_10470483,PGF_10471787,PGF_10474010,PGF_10506571,PGF_10522720,PGF_10535492,PGF_12758906,PGF_12946886
Genome ID,,,,,,,,,,,,,,,,,,,,,
360118.7,0,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,1,0,1,0,0
90239.3,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1392.338,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1801.3,0,0,0,0,1,0,1,0,0,0,...,0,1,0,0,1,1,0,1,0,0
1561.41,0,0,1,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [69]:
pgfams = {}
with open(path / 'PATRIC_pgfams.txt') as f:
    header = f.readline()
    for line in f:
        
        pgfam = line[:12]
        product = line.strip()[13:]
        pgfams[pgfam] = product

In [70]:
class_features = feature_importance.get_top_features_per_class_in_multiple_runs(X_train, y_train, feature_importance.feature_importance, n_runs=100)

hps_df, nhps_df = feature_importance.create_top_feats_df(class_features, X_train, y_train, pgfams, top_feats=15)

In [71]:
hps_df

,Feature,Function,Mean Importance (SD),HPs,NHPs,P-Ratio
0,PGF_04139053,Uroporphyrinogen III decarboxylase (EC 4.1.1.37),0.038 (0.012),362,27,6.67
1,PGF_01915472,Dihydrolipoamide acetyltransferase component of pyruvate dehydrogenase complex (EC 2.3.1.12),0.035 (0.01),385,48,3.99
2,PGF_07629184,Cytosol aminopeptidase PepA (EC 3.4.11.1),0.03 (0.009),366,39,4.67
3,PGF_07157721,"""Heme O synthase, protoheme IX farnesyltransferase, COX10-CtaB""",0.022 (0.007),312,14,11.09
4,PGF_00022550,Molybdopterin synthase catalytic subunit MoaE (EC 2.8.1.12),0.013 (0.005),303,17,8.87
5,PGF_01033770,Dihydroorotate dehydrogenase (quinone) (EC 1.3.5.2),0.011 (0.005),333,35,4.73
6,PGF_00006100,tRNA-modifying protein YgfZ,0.011 (0.006),305,17,8.93
7,PGF_07941512,23S rRNA (uracil(1939)-C(5))-methyltransferase (EC 2.1.1.190),0.01 (0.003),324,37,4.36
8,PGF_00405499,"""YpfJ protein, zinc metalloprotease superfamily""",0.009 (0.003),273,13,10.45
9,PGF_06757295,Threonine dehydratase biosynthetic (EC 4.3.1.19),0.008 (0.004),323,34,4.73


In [72]:
nhps_df

,Feature,Function,Mean Importance (SD),HPs,NHPs,P-Ratio
0,PGF_03029062,"""Dihydroorotate dehydrogenase (NAD(+)), catalytic subunit (EC 1.3.1.14)""",0.034 (0.01),70,182,0.19
1,PGF_01667671,Cytidylate kinase (EC 2.7.4.25),0.031 (0.009),7,125,0.03
2,PGF_02930287,Reverse rubrerythrin,0.027 (0.01),11,125,0.04
3,PGF_08946513,Flavodoxin,0.021 (0.007),67,170,0.20
4,PGF_01469197,"""RNA methyltransferase, TrmA family""",0.021 (0.009),76,180,0.21
5,PGF_01333294,Activator of (R)-2-hydroxyglutaryl-CoA dehydratase,0.019 (0.007),25,128,0.10
6,PGF_01284176,Rubrerythrin,0.018 (0.009),35,142,0.12
7,PGF_00006245,Formate--tetrahydrofolate ligase (EC 6.3.4.3),0.016 (0.007),138,201,0.34
8,PGF_00033940,Phosphoribosylaminoimidazolecarboxamide formyltransferase (EC 2.1.2.3),0.016 (0.006),6,106,0.03
9,PGF_10332317,Electron transport complex protein RnfB,0.014 (0.006),18,123,0.07


In [73]:
old_features = ['PGF_07157721', 
                'PGF_00022550', 
                'PGF_00405499',
                'PGF_06594013',
                'PGF_06757295',
                'PGF_07941512',
                'PGF_00006461',
                'PGF_00037937',
                'PGF_03081665',
                'PGF_02905791',
                'PGF_08199696',
                'PGF_01147190',
                'PGF_07854425',
                'PGF_01761390',
                'PGF_09847065'
               ]

In [74]:
len(set(hps_df['Feature'][:15]).intersection(old_features))

9

# Test according to groups

In [75]:
GROUP = 'Group'
GENOME_ID = 'Genome ID'

In [76]:
test_genome_verified = pd.read_csv('test_genomes_verified.csv', dtype=str, index_col=0)
test_genome_verified.index = test_genome_verified[GENOME_ID]
test_genome_verified.head()

,Genome ID,Genome Name,Label,HP/NHP entire dataset,species,References,Group
Genome ID,,,,,,,
163603.4,163603.4,Actinomadura latina strain ATCC BAA-277,HP,1/0,Actinomadura latina,\cite{trujillo1997polyphasic},HP
648.157,648.157,Aeromonas caviae strain ScAc2001,HP,9/0,Aeromonas caviae,\cite{tang2020co},HP
565.15,565.15,Atlantibacter hermannii strain 3608,HP,1/0,Atlantibacter hermannii,\cite{ioannou2019escherichia},HP
2026190.21,2026190.21,Bacillus mobilis strain 1428.155,HP,1/0,Bacillus mobilis,\cite{2026190.21},HP
29459.655,29459.655,Brucella melitensis strain HN20190002,HP,13/0,Brucella melitensis,\cite{li2020molecular},HP


In [77]:
test_genome_verified[GROUP].value_counts()

HP-OPP     76
NHP        61
NHP-OPP    41
HP         26
Name: Group, dtype: int64

In [84]:
for group_name, genomes in test_genome_verified.groupby(GROUP):
    
    X_test_group = X_test_raw[genomes.index]
    y_test_group = y_test[genomes.index]
    
    y_pred = WSPC.predict(X_test_group)
    correct = sklearn.metrics.accuracy_score(y_test_group, y_pred, normalize=False)
    accuracy = sklearn.metrics.accuracy_score(y_test_group, y_pred)
    
    y_probs = WSPC.predict_proba(X_test_group)[:, 1]
    average_score = np.average(y_probs)
    print(f'{group_name}\taccuracy={accuracy:.2f} correctly predicted={correct}/{len(y_test_group)} average score={average_score:.2f}')


HP	accuracy=1.00 correctly predicted=26/26 average score=0.94
HP-OPP	accuracy=0.92 correctly predicted=70/76 average score=0.90
NHP	accuracy=0.92 correctly predicted=56/61 average score=0.13
NHP-OPP	accuracy=0.61 correctly predicted=25/41 average score=0.43
